# 运输问题 例1

例1 某公司经销甲产品。它下设三个加工厂。每日的产量分别是: A1 为 7 吨, A2
为 4 吨, A3 为 9 吨。该公司把这些产品分别运往四个销售点。各销售点每日销量为: B1 为3吨,B2 为6吨,B3 为5吨,B4 为6吨。已知从各工厂到各销售点的单位产品的运价 为表 3 -3 所示。问该公司应如何调运产品 , 在满足各销点的需要量的前提下 , 使总运费为 最少。


参考：`facility.py` http://www.gurobi.com/documentation/7.0/examples/facility_py.html

### Step 1: Import functions from the gurobipy module

In [1]:
from gurobipy import *


### Step 2: Create empty model

In [2]:
m = Model("facility")

# Warehouse demand in thousands of units
# 销量
demand = [3, 6, 5, 6]

# Plant capacity in thousands of units
# 产量
capacity = [7, 4, 9]

# Fixed costs for each plant
# 无固定支出
fixedCosts = [0, 0, 0, 0]

# Transportation costs per thousand units
transCosts = [[3, 11, 3 , 10],
              [1, 9 , 2 ,  8],
              [7, 4 , 10,  5],
             ]

transCostsDict = {}


### Step 3: Create activitiy variables

In [3]:
# Range of plants and warehouses
plants = range(len(capacity))
warehouses = range(len(demand))

# 将cost矩阵转为字典，预备计算代价
for i in range(len(capacity)):
    for j in range(len(demand)):
        transCostsDict[(i,j)] = transCosts[i][j]
print "cost矩阵", transCosts
print "cost字典()", transCostsDict
##
# facility.py 中 考虑工厂开放存在cost，增加关闭变量
# Plant open decision variables: open[p] == 1 if plant p is open.
# open = m.addVars(plants,
#                  vtype=GRB.BINARY,
#                  obj=fixedCosts,
#                  name="open")
# 
## 我们这里假设所有工厂都在开放状态
open = [1] * len(plants)

# Transportation decision variables: transport[w,p] captures the
# optimal quantity to transport to warehouse w from plant p
# transport = m.addVars(warehouses, plants, obj=transport name="trans")
# 将原有的 obj=transport 修改为 obj 函数； 
# 按原有的写法 obj 不起作用; 与答案第一行一致，但是二三行不一样。 参答案 p. 83 表 3-13
transport = m.addVars(plants, warehouses, name="trans")
transport

cost矩阵 [[3, 11, 3, 10], [1, 9, 2, 8], [7, 4, 10, 5]]
cost字典() {(0, 1): 11, (1, 2): 2, (0, 0): 3, (2, 1): 4, (0, 2): 3, (2, 0): 7, (1, 3): 8, (2, 3): 5, (2, 2): 10, (1, 0): 1, (0, 3): 10, (1, 1): 9}


{(0, 0): <gurobi.Var trans[0,0]>,
 (0, 1): <gurobi.Var trans[0,1]>,
 (0, 2): <gurobi.Var trans[0,2]>,
 (0, 3): <gurobi.Var trans[0,3]>,
 (1, 0): <gurobi.Var trans[1,0]>,
 (1, 1): <gurobi.Var trans[1,1]>,
 (1, 2): <gurobi.Var trans[1,2]>,
 (1, 3): <gurobi.Var trans[1,3]>,
 (2, 0): <gurobi.Var trans[2,0]>,
 (2, 1): <gurobi.Var trans[2,1]>,
 (2, 2): <gurobi.Var trans[2,2]>,
 (2, 3): <gurobi.Var trans[2,3]>}

### Step 4: Set objective function

In [4]:
# 为何这里的 obj=transCosts ?
# transport = m.addVars(warehouses, plants, name="trans")
obj = transport.prod(transCostsDict)
m.setObjective( obj , GRB.MINIMIZE )
# m.modelSense = GRB.MINIMIZE
# m.setObjective(quicksum(transport.prod(transCosts)), GRB.MINIMIZE)
print "目标函数 obj:", obj

目标函数 obj: <gurobi.LinExpr: 3.0 trans[0,0] + 11.0 trans[0,1] + 3.0 trans[0,2] + 10.0 trans[0,3] + trans[1,0] + 9.0 trans[1,1] + 2.0 trans[1,2] + 8.0 trans[1,3] + 7.0 trans[2,0] + 4.0 trans[2,1] + 10.0 trans[2,2] + 5.0 trans[2,3]>


### Step 5: Add constraints

In [5]:
# Production constraints
# Note that the right-hand limit sets the production to zero if the plant
# is closed
# <= 修改为 ==
m.addConstrs(
    (transport.sum(p, '*') == capacity[p] for p in plants),
    "Capacity")

# Demand constraints
m.addConstrs(
    (transport.sum('*', w) == demand[w] for w in warehouses),
    "Demand")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

### Step 6: Solve model

In [6]:
m.optimize()

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 9e+00]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7000000e+01   1.400000e+01   0.000000e+00      0s
       3    8.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds
Optimal objective  8.500000000e+01


### Step 8: Print variable values for optimal solution

In [7]:
# Print solution
print('\nTOTAL COSTS: %g' % m.objVal)
print('SOLUTION: 运输矩阵解为')
print "   ",
for w in warehouses:
    print "[B{0}]".format(w),
print 
for p in plants:
    print "[A{0}]".format(p), 
    if open[p] > 0.99:
        # print('Plant %s open' % p)
        for w in warehouses:
            #if transport[w,p].x > 0:
            #    print('  Transport %g units to warehouse %s' % \
            #          (transport[w,p].x, w))
            print transport[p,w].x, 
        print
    else:
        print('Plant %s closed!' % p)
        
# print transport[0,1].x
# print transport[1,0].x
# print transport[3,0].x
print transport[0,2].x


TOTAL COSTS: 85
SOLUTION: 运输矩阵解为
    [B0] [B1] [B2] [B3]
[A0] 2.0 0.0 5.0 0.0
[A1] 1.0 0.0 0.0 3.0
[A2] 0.0 6.0 0.0 3.0
5.0


```
# 解出的最优解满足生产运输约束, 与答案一致
  [B0] [B1] [B2] [B3]
[A0] 2.0 0.0 5.0 0.0
[A1] 1.0 0.0 0.0 3.0
[A2] 0.0 6.0 0.0 3.0
```

In [8]:
from numpy import array, dot

ar_cost = array(transCosts)
print "运输代价\n", ar_cost
             
# 虽然两个矩阵的结果不一样， 但是计算出来的最优值是一样的，都是 85
ourResult = [[2, 0, 5 , 0],
              [1, 0 , 0 , 3],
              [0, 6 , 0,  3],
             ]
bookResult= [[0, 0, 5 , 2],
              [3, 0 , 0 ,  1],
              [0, 6 , 0,  3],
             ]

ar_our = array(ourResult)
ar_book = array(bookResult)
# reduce(lambda ourResult, transCosts:  ourResult * transCosts, list, 1)  
cost_our=(ar_cost * ar_our)
print "\ncost_our 我们计算 \n", ourResult
print "代价相乘后 \n", cost_our
print "最优值 ", cost_our.sum()
cout_book = ar_cost * ar_book
print
print "cout_book 书本计算\n", bookResult
print "代价相乘后\n", cout_book
print "最优值 ",cout_book.sum()

if cost_our.sum() == cout_book.sum():
    print "\nCongraturations! 最优值一致"

运输代价
[[ 3 11  3 10]
 [ 1  9  2  8]
 [ 7  4 10  5]]

cost_our 我们计算 
[[2, 0, 5, 0], [1, 0, 0, 3], [0, 6, 0, 3]]
代价相乘后 
[[ 6  0 15  0]
 [ 1  0  0 24]
 [ 0 24  0 15]]
最优值  85

cout_book 书本计算
[[0, 0, 5, 2], [3, 0, 0, 1], [0, 6, 0, 3]]
代价相乘后
[[ 0  0 15 20]
 [ 3  0  0  8]
 [ 0 24  0 15]]
最优值  85

Congraturations! 最优值一致
